In [111]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tp2datostrain/test.csv
/kaggle/input/tp2datostrain/train.csv
/kaggle/input/tpdatostest/test.csv


In [112]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [113]:
import pandas as pd
test = pd.read_csv("../input/tp2datostrain/test.csv")
display(test.shape)
train = pd.read_csv("../input/tp2datostrain/train.csv")
display(train.shape)


(60000, 22)

(240000, 23)

In [114]:
def rellenar_con_X_por_grupo(df, grupo, rellenar, func):
    valoresAgrupados = df.groupby(grupo)[rellenar].transform(func)
    df[rellenar].fillna(valoresAgrupados, inplace=True)

### Arreglo algunas cosas del DF
#### Swapeo metros totales y cubiertos cuando los cubiertos son mayores

In [115]:
####Data frame a usar
df = train
df["fecha"] = pd.to_datetime(df["fecha"])

In [116]:
df["swapear"] = df["metrostotales"] < df["metroscubiertos"]
df_metros_arreglados = df.copy()
aux = df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metrostotales"]
if (not aux.empty):
    df_metros_arreglados.loc[df_metros_arreglados["swapear"],"metrostotales"] = \
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"]
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"] = aux
df_metros_arreglados.drop(columns="swapear", inplace=True)
(df_metros_arreglados["metrostotales"] < df_metros_arreglados["metroscubiertos"]).value_counts()
df = df_metros_arreglados

### Arreglo nulos

In [117]:
#RELLENO LOS IDS CON LO QUE MAS APARECE EN LA CIUDAD
def obtenerMasFrecuente(x):
    x = pd.Series.mode(x)
    s = x.loc[x>0]
    if len(s.index)<=0:
        return np.nan
    return s.value_counts().index[0]
print("Nulos idzona: ", df["idzona"].isna().value_counts())
print("Nulos coidad: ", df["ciudad"].isna().value_counts())

aux = df[["ciudad", "idzona"]].copy()
aux["ciudad"].fillna("dummy", inplace=True)
#aux["idzona"].fillna(-1, inplace=True)
idzonaMasFrecuentePorCiudad = aux.groupby('ciudad')["idzona"].transform(lambda x: obtenerMasFrecuente(x))
idzonaMasFrecuentePorCiudad

Nulos idzona:  False    211379
True      28621
Name: idzona, dtype: int64
Nulos coidad:  False    239628
True        372
Name: ciudad, dtype: int64


0         50003995.0
1            24481.0
2            48638.0
3            53666.0
4            47735.0
             ...    
239995       53666.0
239996       51929.0
239997    50003995.0
239998       24448.0
239999       23734.0
Name: idzona, Length: 240000, dtype: float64

In [118]:
df.idzona.fillna(idzonaMasFrecuentePorCiudad, inplace = True)
print("Nulos idzona: ", df["idzona"].isna().value_counts())
#DROPEO LAS COLUMNAS QUE QUEDARON SIN NADA
df.dropna(subset=['idzona'], inplace = True)
print("Nulos idzona: ", df["idzona"].isnull().value_counts())
df.shape

Nulos idzona:  False    239521
True        479
Name: idzona, dtype: int64
Nulos idzona:  False    239521
Name: idzona, dtype: int64


(239521, 23)

In [119]:
#Reemplazo las habitaciones por la mediana de la zona
print("Nulos habitaciones: ", df["habitaciones"].isnull().value_counts())
rellenar_con_X_por_grupo(df, "idzona", "habitaciones", lambda x: x.median())
print("Nulos habitaciones: ", df["habitaciones"].isnull().value_counts())
#Relleno el resto con las mas frecuentes
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

Nulos habitaciones:  False    217110
True      22411
Name: habitaciones, dtype: int64
Nulos habitaciones:  False    238267
True       1254
Name: habitaciones, dtype: int64


In [120]:
#Para la antiguedad relleno con el promedio de la zona
print("Nulos antiguedad: ", df["antiguedad"].isna().value_counts())
medianaAntiguedadXZona = df.groupby("idzona")["antiguedad"].transform(lambda x: x.median())
df["antiguedad"].fillna(medianaAntiguedadXZona, inplace = True)
print("Nulos antiguedad: ", df["antiguedad"].isna().value_counts())

Nulos antiguedad:  False    196234
True      43287
Name: antiguedad, dtype: int64
Nulos antiguedad:  False    237924
True       1597
Name: antiguedad, dtype: int64


In [121]:
#Reemplazo los banos por 1 que es lo minimo posible
df.banos.fillna(1, inplace=True)
#Relleno garages con 0 porque es lo mas probable
df.garages.fillna(0, inplace = True)

In [122]:
print("Nulos metrostotales: ", df["metrostotales"].isna().value_counts())
print("Nulos metroscubiertos: ", df["metroscubiertos"].isna().value_counts())
#COMO QUEDAN NULOS LOS RELLENO CON la mediana (porque la mediana es un valor, y es menos sensible a valores extremos) por ZONA / CIUDAD
promedioMetrosCubiertosXZona = df.groupby("idzona")["metroscubiertos"].transform(lambda x: x.median())
df["metroscubiertos"].fillna(promedioMetrosCubiertosXZona, inplace = True)
print("Nulos metroscubiertos: ", df["metroscubiertos"].isna().value_counts())
promedioMetrosTotalesXZona = df.groupby("idzona")["metrostotales"].transform(lambda x: x.median())
df["metrostotales"].fillna(promedioMetrosCubiertosXZona, inplace = True)
print("Nulos metrostotales: ", df["metrostotales"].isna().value_counts())

Nulos metrostotales:  False    188189
True      51332
Name: metrostotales, dtype: int64
Nulos metroscubiertos:  False    222183
True      17338
Name: metroscubiertos, dtype: int64
Nulos metroscubiertos:  False    238777
True        744
Name: metroscubiertos, dtype: int64
Nulos metrostotales:  False    239521
Name: metrostotales, dtype: int64


#### Agrego precio por metro cuadrado? Esta filtrando datos del precio?

#### Agrego cuantas publicaciones hubo en ese dia


In [123]:
df["nro_publicaciones_dia"] = df.groupby(pd.Grouper(key='fecha', freq='D'))["id"].transform(lambda x: len(x.index))
df["nro_publicaciones_semana"] = df.groupby(pd.Grouper(key='fecha', freq='W'))["id"].transform(lambda x: len(x.index))
df["nro_publicaciones_5_dias_habiles"] = df.groupby(pd.Grouper(key='fecha', freq='B'))["id"].transform(lambda x: len(x.index))
df["nro_publicaciones_mes"] = df.groupby(pd.Grouper(key='fecha', freq='M'))["id"].transform(lambda x: len(x.index))

## Hago train - test split

In [124]:
display(df.shape)
features = [ 'idzona', 'tipodepropiedad', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
            'provincia', 'gimnasio','centroscomercialescercanos','usosmultiples','piscina','escuelascercanas',
           'nro_publicaciones_dia','nro_publicaciones_semana', 'nro_publicaciones_5_dias_habiles', 'nro_publicaciones_mes']
train_y = df.precio.copy()
train_x = df[features].copy()
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y,train_size=0.2, test_size=0.8, random_state = 0)

(239521, 27)

In [125]:
transformador_numeros = SimpleImputer() #usa promedio
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
cols_numericas = [ 'idzona', 'antiguedad', 'habitaciones', 'garages', 'banos',
                  'metroscubiertos', 'metrostotales',
             'gimnasio','centroscomercialescercanos','usosmultiples',
                  'piscina','escuelascercanas',
           'nro_publicaciones_dia','nro_publicaciones_semana', 'nro_publicaciones_5_dias_habiles',
                  'nro_publicaciones_mes']

cols_categoricas = ['tipodepropiedad', 'provincia']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, cols_numericas),
        ('cat', transformador_categorias, cols_categoricas)
    ],remainder = "drop")#remainder = "passthrough"

In [126]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=12, subsample_by_tree=0.5, colsample_by_col=0.5, min_child_weight=0,
                     colsample_bytree = 0.7747100973226512, x_gamma = 0.47299983077893815,
                     x_reg_lambda = 0.12707727377560518, x_subsample = 0.7361630765924525,
                     objective='reg:squarederror',
                     tree_method='gpu_hist', gpu_id=0)
from catboost import CatBoostRegressor
model2 = CatBoostRegressor(iterations=700, depth=7, learning_rate=0.05, loss_function='RMSE', task_type="GPU")

In [127]:
pipeline2 = make_pipeline(preprocessor, model, verbose = True)

### Optimizacion de parametros
Despues de 30 iteraciones me devolvió lo siguiente:  
`{'x_colsample_bytree': 0.7747100973226512, 'x_gamma': 0.47299983077893815, 'x_max_depth': 12.0, 'x_min_child': 9.0, 'x_reg_lambda': 0.12707727377560518, 'x_subsample': 0.7361630765924525}`

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import accuracy_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import numpy as np

X, Xcv, y, ycv = train_test_split(train_x,train_y,test_size = 0.2,random_state=0)

def objective(space):
    # Instantiate the classifier con los nuevos hyperparametros
    clf = XGBRegressor(n_estimators =1000,colsample_bytree=space['colsample_bytree'],
                           learning_rate = .3,
                            max_depth = int(space['max_depth']),
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                           gamma = space['gamma'],
                           reg_lambda = space['reg_lambda'],
                            objective='reg:squarederror',
                           tree_method='gpu_hist', gpu_id=0)
    
    eval_set  = [( X, y), ( Xcv, ycv)]
    
    pipe = make_pipeline(preprocessor, clf, verbose = True)
    # Fit the classsifier
    pipe.fit(X, y)
    
    # Predict on Cross Validation data
    pred = pipe.predict(Xcv)
    
    # Calculate our Metric - accuracy
    accuracy = accuracy_score(ycv, pred>0.5)

    # return needs to be in this below format. We use negative of accuracy since we want to maximize it.
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
#Defino mis posibles hiperparametros
#Notese que usa distribuciones para definir los hiperparametros
space ={'max_depth': hp.quniform("x_max_depth", 4, 16, 1),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.7, 1),
        'gamma' : hp.uniform ('x_gamma', 0.1,0.5),
        'colsample_bytree' : hp.uniform ('x_colsample_bytree', 0.7,1),
        'reg_lambda' : hp.uniform ('x_reg_lambda', 0,1)
    }

In [ ]:
#Busco el mejor
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials)
print(best)

In [128]:
#my_pipeline.fit(train_x, train_y)
pipeline2.fit(train_x, train_y)

[Pipeline] . (step 1 of 2) Processing columntransformer, total=  36.3s


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[Pipeline] ...... (step 2 of 2) Processing xgbregressor, total= 1.3min


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0),
                                                  ['idzona', 'antiguedad',
                                                   'habitaciones', 'garages',
                                                   'banos', 'met

In [106]:
#preds = my_pipeline.predict(test_x)
preds2 = pipeline2.predict(train_x)

In [107]:
display(preds2.shape)
display(test_x.shape)

(59861,)

NameError: name 'test_x' is not defined

In [85]:
from sklearn.metrics import mean_absolute_error
#print("MAE XGBoost: ", mean_absolute_error(test_y, preds))
print("MAE CatBoost: ", mean_absolute_error(test_y, preds2))

NameError: name 'test_y' is not defined

In [87]:
from sklearn.model_selection import cross_val_score
SCORE = "mean_absolute_error"
def cross_validation(pipeline, X, y, nro_folds):
    score = -1 * cross_val_score(pipeline, X, y,
                              cv=nro_folds,
                              scoring='neg_mean_absolute_error')
    print("Puntajes: ", end="")
    for p in score:
        print(p, ", ", end="")
    
    print("\nPromedio: ", np.mean(score))
    
cross_validation(pipeline2, train_x, train_y, 2)
#cross_validation(my_pipeline, train_x, train_y, 5)

[Pipeline] . (step 1 of 2) Processing columntransformer, total=   9.2s


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[Pipeline] ...... (step 2 of 2) Processing xgbregressor, total= 1.1min
[Pipeline] . (step 1 of 2) Processing columntransformer, total=   8.9s


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[Pipeline] ...... (step 2 of 2) Processing xgbregressor, total= 1.1min
Puntajes: 592602.9876080016 , 597415.6883835053 , 
Promedio:  595009.3379957534


### GRafico importancia

In [133]:
import xgboost as xgb
xgb.plot_importance(model)

ValueError: tree must be Booster, XGBModel or dict instance

In [ ]:
test.shape

In [99]:
res = pd.DataFrame(preds2, index=test.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)

ValueError: Shape of passed values is (59861, 1), indices imply (60000, 1)